In [139]:
import numpy as np
import pandas as pd
import re
import random
import requests
from datasets import load_dataset
import spacy

In [3]:
random.seed(2137)

In [4]:
df_texts = pd.DataFrame.from_dict(load_dataset("clarin-knext/fiqa-pl", "corpus")['corpus'])
df_texts.head(2)

,_id,title,text
0,3,,"Nie mówię, że nie podoba mi się też pomysł szk..."
1,31,,Tak więc nic nie zapobiega fałszywym ocenom po...


In [5]:
df_qa = pd.DataFrame(load_dataset("clarin-knext/fiqa-pl-qrels")['test'])
df_qa.head(2)

,query-id,corpus-id,score
0,8,566392,1
1,8,65404,1


In [6]:
df_q = pd.DataFrame(load_dataset("clarin-knext/fiqa-pl", "queries")['queries'])
df_q.head(2)

,_id,title,text
0,0,,Co jest uważane za wydatek służbowy w podróży ...
1,4,,Wydatki służbowe - ubezpieczenie samochodu pod...


In [7]:
# 1000 random passages:
df_1000_passages = df_texts.sample(n=1000, random_state=2137)
df_1000_passages.info(2)

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 47960 to 23135
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     1000 non-null   object
 1   title   1000 non-null   object
 2   text    1000 non-null   object
dtypes: object(3)
memory usage: 31.2+ KB


# Setting up Ollama after isntalling it locally:

In [8]:
!ollama list

NAME               ID              SIZE      MODIFIED   
llama3.2:latest    a80c4f17acd5    2.0 GB    4 days ago    


In [9]:
!ollama serve

Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [10]:
def query_llama(prompt, model="llama3.2"):
    url = "http://localhost:11434/api/chat"
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(url, json=payload, headers=headers, stream=True)
    
    if response.status_code != 200:
        print(f"Błąd {response.status_code}: {response.text}")
        return None

    full_response = ""
    for line in response.iter_lines():
        if line:
            try:
                json_line = line.decode('utf-8')
                data = requests.models.complexjson.loads(json_line)
                full_response += data.get("message", {}).get("content", "")
            except ValueError:
                print(f"Niepoprawny JSON: {line}")
    
    return full_response

prompt = "Mówisz po polsku?"
result = query_llama(prompt)
print(result)

Tak, mówię po polsku. Co mogę dla Ciebie zrobić? Chcesz pomóc mi z językiem lub mieć konwersację?


# Test spacy:

In [11]:
nlp = spacy.load("pl_core_news_sm")

In [12]:
print(nlp.get_pipe("ner").labels)

('date', 'geogName', 'orgName', 'persName', 'placeName', 'time')


O klasyfikacje do tych samych kategorii będę prosił LLamę, oczywicie w celu porównania wyników.

In [13]:
# test NER spacy:

In [14]:
def lemmatize_text(text):
    doc = nlp(text)
    lematized_words = [token.lemma_ for token in doc]
    return " ".join(lematized_words)

In [15]:
df_1000_passages['text_lemmat'] = df_1000_passages['text'].apply(lemmatize_text)

In [16]:
def identify_entities(document):
    doc = nlp(document)
    entities_labels = [(ent.text, ent.label_) for ent in doc.ents]
    return entities_labels
df_1000_passages['expressions_spacy'] = df_1000_passages['text_lemmat'].apply(identify_entities)

In [21]:
df_1000_passages['expressions_spacy'].iloc[12:20]

19231                                                   []
16355                                                   []
3985                                     [(Y ., persName)]
24905    [(GBP, orgName), (2010 rok, date), (1265, date...
15322                                [(Kleenex, persName)]
45428    [(Cincinnati, placeName), (OH Pros, orgName), ...
41069             [(Guys, placeName), (Culver, placeName)]
34881    [(The Weinstein Company, orgName), (Hollywood,...
Name: expressions_spacy, dtype: object

## Zero-Shot comparison

In [22]:
def identify_entities_one_shot(text):
    query = f"""
    W tekście poniżej wyszukaj Nazwanych Encji. Zaklasyfikuj je do jednej z 6 kategorii:
    'date' - data 
    'geogName' - Miejsce georaficzne
    'orgName' - Nazwa firmy
    'persName' - Nazwa personalna
    'placeName' - Nazwa miejsca
    'time' - czas
    Przeanalizuj ten tekst:{text}
    Zwróć jedynie listę krotek w postaci: [(encja1, kategoria1), (encja2, kategoria2) ...]
    Analizuj TYLKO I WYŁACZNIE POWYZSZY TEKST i ZWRÓC TYLKO LISTE WYNIKOWA
    """
    return query_llama(query)

In [23]:
def compare_llama_spacy(query_idx):
    print("TEKST:")
    print(df_1000_passages['text_lemmat'].iloc[query_idx])
    print("LLama:")
    print(identify_entities_one_shot(df_1000_passages['text_lemmat'].iloc[query_idx]))
    print("Spacy:")
    print(df_1000_passages['expressions_spacy'].iloc[query_idx])

In [24]:
compare_llama_spacy(123)

TEKST:
„ to dobry tl;dr , jaki móc być stworzyć , [ oryginalny](http://mobile.reuters.c / article / amp / idUSKBN1AD0DU ) zmniejszć o 83 % . ( być bot ) * * * * * > WASZYNGTON / LONDYN WASZYNGTON / LONDYN – eksport węgiel z USA wzrosnąć w ten rok o ponad 60 procent z powód rosnąć popyt w Europa i Azja , według przegląd dane rządowy Reuter , co pozwolić administracja prezydent Donald Trump na stwierdzić , że wysiłek aby ożywić zniszczona przemysł . > „ jeśli Europa chcieć pouczać Trump o klimat , państwo członkowski UE potrzebować plan przejściowy , aby stopniowo wycofywać się z zanieczyszczający węgiel ” – powiedzieć Laurence Watson , analityk dane pracować nad węgiel w niezależny think tanek Carbon Tracker . inicjatywa w Londyn > ZACIĄGNIĘCIE KREDYTÓW zarówno przemysł węglowy , jak i administracja Trumpa stwierdzić , że rosnący eksport zarówno węgiel energetyczny , używać do wytwarzać energia elektryczny , jak i węgiel metalurgiczny , używać w przemyśł ciężki , być dowód na to , że pr

Spacy znacznie wiecej znalezionych encji. Oba dobrze poradziły sobie z kategoria persName. Generalnie spacy lepiej


In [26]:
compare_llama_spacy(13)

TEKST:
w kontekst ten artykuł to milion dolar . ale mieć racja , wszystek ubezpieczenie na życie ten oszustwo , nawet jeśli nie zawsze być to milion dolar . mój żona nie ucierpieć finansowo , jeśli umrę . być silny , niezależny kobieta . mieć dobry kariera .
LLama:
Wynik:
[(orgName, 'date'), (persName, 'date'), (geogName, 'date')]
Spacy:
[]


Llama cos znalazla, blednie, spacy nic i poprawnie. Spacy lepiej

In [27]:
compare_llama_spacy(66)

TEKST:
Zacznij od wkładać pieniądz w złoty moneta kruszcowy : jeśli być początkujący , upewnić się , że zaczynać od wkładać pieniądz w złoty moneta kruszcowy . być bardzo prosty w obsługa , w to wiązać się z bardzo minimalny ryzykie . po zakup złoto kruszcowy lub złoty moneta otrzymać to sam od dowolny firma kurierski .
LLama:
[(persName, 'date'), 
 ('geogName', 'placeName')].
Spacy:
[('ryzykie', 'placeName')]


Llama halucynuje, spacy cos znalazlo niestety zle, oba blednie.

In [28]:
compare_llama_spacy(132)

TEKST:
jeśli nie mieć osoba na utrzymanie , nie mieć potrzeba ubezpieczenie na życie . wspominać o wykupiyć ubezpieczenie , gdy nie być on potrzebny , aby uchronić ty przed jakiś przyszły ryzyko . jeśli mieć polisę i pojawić się choroba , który normalnie uniemożliwiać by ubezpieczenie , móc zachować ubezpieczenie do koniec okres . koszt być by bardzo wysoki . musiałbyś mieć termin , który trwać by dziesięciolecie , aby objąć ty , dopóki jakiś przyszły dziecko nie opuścić college'u . jeśli nigdy nie mieć ktoś , kto zależeć od Ciebie , jeśli chodzić o dochód , nigdy nie mieć potrzeba ubezpieczenie na życie .
LLama:
[(Nazwa Encji 1, 'persName'), (Nazwa Encji 2, 'time')]

Te dwa Nazwy Encji to: Janusz Korczak
Spacy:
[('musiałbyś', 'persName')]


Oba zle, LLama halucynuje

In [29]:
compare_llama_spacy(997)

TEKST:
nie zgadzać się . w początkowy faz II wojna światowy mieć być znacznie mniej dług niż teraz . nie mieć wątpliwość , że gdyby być mieć duży dług , to uwzględnić by to nasz wkład w wysiłek wojenny , tak jak nasz obecny poziom zadłużenie być uwzględnić w nasz udział w koszyk gówniany beczka proch , który być na Bliski Wschód . teraz . to mój punkt ! nie móc stworzyć nieskończonych pieniądz , więc jeśli już stworzyć być gówniany tona pieniądz , wpłynie to na twój decyzja o stworzyć kolejny gówniany tona pieniądz ! nie wiedzieć , czy to słomiać człowiek , czy nie , ale oba uważać , że rząd móc więcej . jeśli nie zgadzać się z ja , że istnieć ograniczenie co do to , ile rząd móc wydać , to szczerze mówić , być po prosty z strata . chcieć wiedzieć , konkretnie i precyzyjnie , jaki być limit w czas rzeczywisty , aby być móc odpowiednio dostosować nasz reakcja polityczny
LLama:
[(Wojna Światowa II, 'date'), (Bliski Wschód, 'geogName')].
Spacy:
[('Bliski Wschód', 'geogName')]


rejon znaleziony poprawnie, LLama co ciekawe dodatkowo znalazla info o wojnie i przypisala mu czas. Mozna powiedziec ze jest ok


In [30]:
compare_llama_spacy(112)

TEKST:
zapotrzebowanie na kariera STEM w 2020 rok wzrosnąć w stosunek do dzisiejszy potrzeba o około 50 % ( departament , 2015 ) . na obszar otaczający szkoła istnieć wiele firma , firma i stowarzyszenie , który w swój codzienny proces mieć aspekt STEM nauka Technologi inżynieria Matematyk . jeszcze przed inicjatywa STEM wiele szkoła podjąć własny wysiłek na rzecz poprawa edukacja w ten dziedzina . edukacja STEM być ważny w konkurencyjny gospodarka światowy , a Stany Zjednoczone pozostawać w tyle w kształceć młodzież . Odwiedź strona http://www.techjoyntfranchise.com/ , aby znaleźć edukacja wyznaczć przez STEM w Stany Zjednoczone .
LLama:
[(Stem, 'orgName'), (szkoła, 'geogName'), (firma, 'geogName'), (stowarzyszenie, 'geogName')]

w tym:
- 'orgName' to nazwa firmy,
- 'geogName' to miejsce georaficzne,
- 'persName' nie ma występu w tekście,
- 'placeName' nie ma występu w tekście,
- 'date' nie ma występu w tekście,
- 'time' nie ma występu w tekście.
Spacy:
[('STEM', 'orgName'), ('2020 ro

LLama halucynuje. Nie trzyma sie standardu outputu. Ciezko nad nia zapanowac. Spacy niby lepiej kilka poprawnych wyszukan, jednak niektore tez niepoprawne

# Wniosek co do Zero-Shot:
Coś znajduje, coś halucynuje, coś jest niepoprawne. Generalnie LLM średnio póki co. Przy zero-shot (LLama 3.2 mam na myśli) wolałbym jednak Ner ze spacy. Spacy szybsze i trafniejsze poki co.

# Few-shot prompting

In [31]:
def identify_entities_few_shot(text):
    query = f"""
    W tekście wyszukaj Nazwanych Encji. Zaklasyfikuj je do jednej z 6 kategorii:
    'date' - data 
    'geogName' - Miejsce georaficzne
    'orgName' - Nazwa firmy
    'persName' - Nazwa personalna
    'placeName' - Nazwa miejsca
    'time' - czas
    
    Przykład 1:
    Text: Lech Wałęsa był prezydentem Polski a teraz już nie jest teraz nim jest Andrzej Duda."
    Output: [("Lech Wałęsa", "persName"), ("Polska", "geogName"), ("Andrzej Duda", "persName")]

    
    Przykład 2:
    Text: "Byłem ostatnio w Stanach Zjednoczonych. Po powrocie poważnie rozważam zainwestowanie w akcje Apple. Myśle że w 2025 roku wystrzęlą do góry."
    Output: [("Stany Zjednoczone", "geogName"), ("Apple", "orgName"), ("2025", "date")]
    
    Przykład 3:
    Text: "Mikołaj Kopernik był astronomem i matematykiem, który urodził się w Toruniu."
    Output: [("Mikołaj Kopernik", "persName"), ("Toruń", "placeName")]
    
    teraz przetwórz tekst:
    
    Text: {text}
    Output: 
    """
    return query_llama(query)

In [32]:
def compare_llama_spacy_few(query_idx):
    print("TEKST:")
    print(df_1000_passages['text_lemmat'].iloc[query_idx])
    print("LLama (few-shot):")
    print(identify_entities_few_shot(df_1000_passages['text_lemmat'].iloc[query_idx]))
    print("Spacy:")
    print(df_1000_passages['expressions_spacy'].iloc[query_idx])

In [33]:
compare_llama_spacy_few(12)

TEKST:
to właściciel zarządzać rezerwacja . mieć tylko skrawek papier z bazgroł . plan dezercja być trudny , kiedy być być na studia . Wyjście na ładniejszy kolacja być duży okazja . dzisiaj nie być by aż tak wielki problem , żeby po prosty wpłacić kaucja .
LLama (few-shot):
Oto wyniki wyszukiwania Nazwanych Encji w tekstie:

1. "Właściciel" - "persName"
2. "Toruń" - "geogName"

Tekst po przetworzeniu może być następujący:
 
    "Właściciel zarządzać rezerwacja mieć tylko skrawek papier z bazgroł plan dezercja być trudny kiedy być na studia wyjście na ładniejszy kolacja będzie duży okazja dzisiaj nie będzie by aż tak wielki problem żeby po prosty wpłacić kaucja"
Spacy:
[]


Spacy nic (dobrze), Llama blednie, halucynuje

In [34]:
compare_llama_spacy_few(23)

TEKST:
„ standardowy darowizna polityczny dla kandydat federalny być ograniczyć do 2700 dolar_amerykański na kandydat na wybory . wybory prawybora i wybory powszechny być w ten cel różny wybory . Źródło : http://www.fec.gov/pages/brochures/citizens.shtml nie mieć podatek implikacja dla wkład w kampania . nawet jeśli wnosić wkład w kampania ktoś , kto wręczać być prezent teraz lub w przeszłość , to się liczyć . Wnosisz wkład w kampania , a nie osoba . taki pieniądz musieć być przeznaczyć na kampania Kandydat móc by zostać oskarżony ( o coś taki jak defraudacja ) za wykorzystać fundusz na coś inny . przykładowy źródło : http://www.rothcpa.com/archives/006985.php sam kongres nakazać IRS odstąpić od bezpośredni datk polityczny przez wprowadzić w życie to , co być obecnie sekcja 2501(c)(4 ) kodeks w 1975 rok . , który zakazywać naliczać podatek od darowizn dla „ „ organizacja polityczny ” ” określić w sekcja 527 jako „ ... partia , komitet , stowarzyszenie , fundusz lub inny organizacja ( n

Spac jedno poprawnie, jedno zle. LLama jedno i to blednie

In [35]:
compare_llama_spacy_few(77)

TEKST:
istnieć fundusz Indeksu społeczny iShares Jantzi .
LLama (few-shot):
Aby wyszukać Nazwy Encji w tekście, będę używał następującego skryptu Pythona:

```python
import re

def wyszukajNazwyEncji(text):
    nazwyEncji = []
    
    # Reguły wyszukiwania
    reguły = [
        r"(\w+)\s+([a-z\s]+)",  # Nazwa personalna
        r"([a-z\s]+)\s+jantzi",  # Nazwa funduszu iShares Jantzi
        r"(indeks społeczny|spółka iShares jantzi)"  # Nazwa funduszu Indeksu społeczny iShares Jantzi
    ]
    
    for reguła in reguły:
        match = re.search(reguła, text)
        
        if match:
            nazwaEncji = match.group(1) or match.group(2)
            kat = None
            
            # Kategoria na podstawie treści nazwy Encji
            if re.match(r"([a-zA-Z]+)\s+([a-zA-Z\s]+)", nazwaEncji):
                kat = 'geogName'
            elif re.match(r"(\w+)\s+\S+", nazwaEncji):
                kat = 'persName'
            else:
                kat = 'orgName'
              

Llama jeden dobrze, jedno halucynacja

In [36]:
compare_llama_spacy_few(113)

TEKST:
jeśli nie wymyślić sposób na zwiększyć monetyzacja swój baza użytkownik , FB nie być mieć powód do wzrost wartość . chociaż obecnie definiować sektor sieć społecznościowy i dobrze się przygotowywać , aby utrzymać ten status w przyszłość , to niewiele znaczyć , jeśli on użytkownik nie kliknić ten reklama .
LLama (few-shot):
Oto wyniki wyszukiwania Nazwanych Encji w tekstach:

Przykład 1:
Text: jeśli nie wymyślić sposób na zwiększenie monetyzacja swój baza użytkownik , FB nie być mieć powód do wzrost wartość .
Output: [("FB", "geogName")]

Przykład 2:
Text: jeśli nie wymyślić sposób na zwiększenie monetyzacja swój baza użytkownik , FB nie będą mieć powód do wzrost wartość .
Output: [("FB", "geogName")]

Przykład 3:
Text: jeśli nie wymyślić sposób na zwiększenie monetyzacji swój baza użytkownik , fb nie będzie mieć powodu do wzrostu wartości .
Output: [("fb", "persName")]
Spacy:
[('FB', 'orgName')]


Spacy i Llama poprawnie. jedna encja i dobra kategoria. Pierwszy przyklad gdy obie metody sa zgodne.

Wnioski:
Llama zadziałała gorzej niż moje oczekiwania. Do tej pory Spacy dawał sobie radę, nie zawsze idealnei lecz powiediząłbym, że lepiej. Bardzo ważny jest tuaj wybór LLM, z pewnością większe modele działały by lepiej. Model taki jak GPT działałby znacznie lepiej, (testowałem w API), jednak jesli mowa o mniejszych LLM do tej pory, uważam, że spacy to lepszy wybór. Można by wytestować polski LLM taki jak bielik, był on trenowany na polskich encjach to pewnie radziłby sobie lepiej. Bielika jednak już testowałem w innym lab, a LLamy nie, stąd decyzja o Llamie. Co ważne, spacyy działa dużo dużo szybciej, to kolejny plus. Dodatkowo LLama nieraz nie trzyma sie struktury zwracanego oututu i wymagaloby to dodatkowego parsowania. Jak widac LLM cos tam wnioskuje cos tam 'mysli' jednak gotowe rozwiazanie ze spacy poki co prowadzi.

# Annotacje reczne

In [51]:
df = pd.read_excel('manual_annot.xlsx')
df.drop(0, axis=0, inplace=True)
df.columns = ['text', 'labels']

In [70]:
df.head(6)

,text,labels
1,"Niemal dwa tygodnie temu doszło do walki, na k...","[('Jake Paul', 'PER', (119, 128)), (""Mike'a Ty..."
2,Te problemy widać już na początku. Ridley Scot...,"[('Ridley Scott', 'PER', (35, 47)), ('Acacius'..."
3,Phoenix Suns po dobrym starcie sezonu ostatnio...,"[('Phoenix Suns', 'ORG', (0, 12)), ('Arizony',..."
4,Oświadczenie Karola Nawrockiego: Szef Instytut...,"[('Karola Nawrockiego', 'PER', (13, 31)), ('In..."
5,Brandin Podziemski występuje w Golden State Wa...,"[('Brandin Podziemski', 'PER', (0, 18)), ('Gol..."
6,W tym wpisie skupimy się na podstawowym wprowa...,"[('Langchain', 'ORG', (116, 125)), ('JS', 'ORG..."


In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 1 to 97
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   text              97 non-null     object
 1   labels            97 non-null     object
 2   labels_formatted  97 non-null     object
 3   spacy_labels      97 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


In [166]:
# ('date', 'geogName', 'orgName', 'persName', 'placeName', 'time')
def refactor_labels(labels):
    categories = {
        "PER" : "persName",
        "ORG" : 'orgName',
        "DATE" : 'date',
        "date" : 'date',
        "Date" : 'date',
        "GPE": 'placeName',
        "time": 'time',
        "PERS": 'persName',
        "TIME": 'time',
        "LOC" : 'placeName'
        
    }
    new_labels = []
    for tpl in eval(labels):
        value, category, idxs = tpl
        if category in categories.keys():
            new_labels.append((value, categories[category]))
    return new_labels

In [167]:
df['labels_formatted'] = df['labels'].apply(refactor_labels)

In [114]:
df.head(3)

,text,labels,labels_formatted,spacy_labels
1,"Niemal dwa tygodnie temu doszło do walki, na k...","[('Jake Paul', 'PER', (119, 128)), (""Mike'a Ty...","[(Jake Paul, persName), (Mike'a Tysona, persNa...","[(bokserski, placeName), (AT&T Stadium, geogNa..."
2,Te problemy widać już na początku. Ridley Scot...,"[('Ridley Scott', 'PER', (35, 47)), ('Acacius'...","[(Ridley Scott, persName), (Acacius, persName)...","[(Ridley Scott, persName), (rzymskiej, placeNa..."
3,Phoenix Suns po dobrym starcie sezonu ostatnio...,"[('Phoenix Suns', 'ORG', (0, 12)), ('Arizony',...","[(Phoenix Suns, orgName), (Arizony, placeName)...","[(Suns, geogName), (Beala, orgName), (Kevina D..."


In [118]:
df['spacy_labels'] = df['text'].apply(identify_entities)

In [119]:
df['llama_labels'] = df['text'].apply(identify_entities_few_shot)

In [170]:
def evaluate_spacy(true_labels, preds):
    true_labels = true_labels

    preds = preds
    TP = 0
    FN = 0
    FP = 0
    for pred in preds:
        if pred in true_labels:
            TP += 1
        if pred not in true_labels:
            FP += 1
    for true_label in true_labels:
        if true_label not in preds:
            FN += 1
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    return precision, recall 

In [171]:
df['spacy_precision'], df['spacy_recall'] = zip(*df.apply(lambda row: evaluate_spacy(row['labels_formatted'], row['spacy_labels']), axis=1))

In [138]:
df['llama_labels'].iloc[1]

'Oto wyniki wyszukiwania Nazwanych Encji w tekście:\n \n [("Ridley Scott", "persName"), ("Acacius", "persName"), ("Pedro Pascal", "persName")]\n\nTe wyniki zostały sklasfikowane w następujących kategoriach:\n\n [("Ridley Scott", \'persName\'), (\'Acacius\', \'persName\')]\n\n("Pedro Pascal" - \'persName\')\n\nzawartych w treści Tekstu.'

In [162]:
def evaluate_llama(true_labels, llama_output):
    true_labels = true_labels
    pattern = r'\("([^\"]+)", \"([^\"]+)\"\)'
    preds = re.findall(pattern, str(llama_output))    
    TP = 0
    FN = 0
    FP = 0
    for pred in preds:
        if pred in true_labels:
            TP += 1
        if pred not in true_labels:
            FP += 1
    for true_label in true_labels:
        if true_label not in list(preds):
            FN += 1
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    return precision, recall 

In [163]:
df['llama_precision'], df['llama_recall'] = zip(*df.apply(lambda row: evaluate_llama(row['labels_formatted'], row['llama_labels']), axis=1))

[('Mikołaj Kopernik', 'persName'), ('Toruń', 'placeName'), ('Stany Zjednoczone', 'geogName'), ('2025', 'date')]
[('Ridley Scott', 'persName'), ('Acacius', 'persName'), ('Pedro Pascal', 'persName')]
[('Phoenix Suns', 'geogName'), ('Beal', 'persName'), ('Kevin Durant', 'persName'), ('Arietyna', 'orgName')]
[('Data', 'time'), ('Miejsce geograficzne', 'geogName'), ('Firma', 'orgName'), ('Nazwa personalna', 'persName'), ('Miejsce', 'placeName')]
[('Brandin Podziemski', 'persName'), ('Golden State Warriors', 'geogName'), ('2023', 'date')]
[]
[('Krzysztof Bosak', 'persName'), ('Collegium Humanum', 'geogName')]
[('Lech Wałęsa', 'persName'), ('Andrzej Duda', 'persName'), ('Stany Zjednoczone', 'geogName'), ('Apple', 'orgName'), ('2025', 'date'), ('Mikołaj Kopernik', 'persName'), ('Toruń', 'placeName')]
[('Lexus', 'orgName')]
[('Bitcoin', 'geogName'), ('Algieria', 'geogName'), ('Boliwia', 'geogName'), ('Macedonia Północna', 'geogName'), ('Indie', 'geogName'), ('Chiny', 'geogName'), ('Wietnam', 'g

In [161]:
print("LLama precision:", df['llama_precision'].mean())
print("LLama recall:", df['llama_recall'].mean())

LLama precision: 0.13802186291573798
LLama recall: 0.1817367340563217


In [164]:
print("Spacy precision:", df['spacy_precision'].mean())
print("Spacy recall:", df['spacy_recall'].mean())

Spacy precision: 0.48917302628642834
Spacy recall: 0.49224727162871496


# Wnioski
Zgodnie z wynikami które były spradzane wcześniej. Spacy poradziło sobie lepiej i to o wiele. Należy mieć na uwadze, ze traktowalem dopasowanie jako porpanwe tylko wtedy gdy jest ono idealne, LLama nieraz potrafila obciac czesc Encji, na przyklad z imienia i nazwizska zwracala tylko imie lub przekrecic literke/odmiane slowa. To wszystko rzutuje na wynik. Nie mniej jednak wniosek z pierwszych obserwacji potwierdzil się. Zbior danych byl bardzo maly bo zaledwie 97 tekstów, w niektórych ktegoriach encji brak było rekordów stąd licznie F1 nie ma zbytniego sensu. Z całą pewnościa wynik zalezy mocno (o ile nie najmocniej) od doboru LLM, wieksze (wiecej parametrow) LLM beda radzily sobie zazwyczaj lepiej. Ponadto sposob dopasoania i czy uwazamy zwrocoan krotke za poprawna czy nie tez rzutuje na wynik, czy interesuje nas tylko kompletne dopasowanie, czy moze byc inna odmiana lub czesc encji. To zalezy od specyfiki tego czego szukamy. Podsumowujac co lepsze? - W przykladzie ktory zasotoswoalem Spacy i to znacznie. Niemniej warto LLM miec na uwadze, moga sie przydac do innych zadan.  

# Questions
1. How does the performance of LLM-based NER compare to traditional approaches? What are the trade-offs in terms of accuracy, speed, and resource usage?
LLM - znacznie wolniejsze, halucynuje, gorsza dokladnosc. Zalezy od LLM, wiekszy model bedzie dzialal dokladniej, ale jeszcze wolniej. Jako zalete mozna podac to ze nie trzeba programowac pipeline tylko 'wytlumaczyc' w prompcie czego oczekujemy na 'outpucie'
2. Which prompting strategy proved most effective for NER and classification tasks? Why?
few-shots byla nieco lepsza, jednak wciaz slaba. Z pewnoscia fin-tuning bylby lepszey jednak wymaga juz dosc sporo danych.
3. What are the limitations and potential biases of using LLMs for NER and classification?
Halucynajce, duze wymagania sprzetowe, zmiany fleksji/form zwracanych encji. Ponadto model nieraz zwraca nieustrukteryzowany output mimo tego samego prompta
4. In what scenarios would you recommend using traditional NER vs. LLM-based approaches?
NER w zastosowaniach gdzie mamy mniej mocy obliczneiowej i moze nieco bardzie ustrukturyzowany tekst. LLM moga sie przydac tam gdzie nie zalezy nam na idealnym dopasowaniu, mamy duza moc obliczeniowa oraz nie chcemy programowac NER a jedynie wrzucic do prompta instrukcje